# Predicting Solubility from Molecular Structure with Regression

- Delaney, J.S., J. Chem. Inf. Comput. Sci. 2004, 44, 3, 1000–1005

In [1]:
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors

from tools import helpers as h

COLUMN_NAMES = ['iupac', 'log_solubility', 'log_solubility_pred', 'SMILES']
sol_data = pd.read_csv('data/delaney.csv', names=COLUMN_NAMES, header=0)
sol_data.head()

,iupac,log_solubility,log_solubility_pred,SMILES
0,"1,1,1,2-Tetrachloroethane",-2.18,-2.794,ClCC(Cl)(Cl)Cl
1,"1,1,1-Trichloroethane",-2.00,-2.232,CC(Cl)(Cl)Cl
2,"1,1,2,2-Tetrachloroethane",-1.74,-2.549,ClC(Cl)C(Cl)Cl
3,"1,1,2-Trichloroethane",-1.48,-1.961,ClCC(Cl)Cl
4,"1,1,2-Trichlorotrifluoroethane",-3.04,-3.077,FC(F)(Cl)C(F)(Cl)Cl


In [2]:
sol_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1144 entries, 0 to 1143
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   iupac                1144 non-null   object 
 1   log_solubility       1144 non-null   float64
 2   log_solubility_pred  1144 non-null   float64
 3   SMILES               1144 non-null   object 
dtypes: float64(2), object(2)
memory usage: 35.9+ KB


Solubilities are measured in mol/L

In [ ]:
target = sol_data.iloc[:, 1].to_numpy()
smiles = list(sol_data.SMILES)

molecules = [Chem.MolFromSmiles(smile) for smile in smiles]

assert len(molecules) == len(target) == len(smiles)

In [ ]:
sample_molecules = molecules[0:3]
for i, mol in enumerate(sample_molecules):
    print("IUPAC:", sol_data['iupac'][i])
    print("SMILES:", sol_data['SMILES'][i])
    print(Chem.MolToMolBlock(mol))
    display(mol)

In [ ]:
sample_mol = molecules[4]

def get_predictors(mol):
    
    molecular_weight = Descriptors.MolWt(mol)
    oct_water_partition_coefficient = Descriptors.MolLogP(mol)
    num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)
    aromatic_proportion = get_aromatic_proportion(mol)
    
    entry = {
        'MW': molecular_weight,
        'cLogP': oct_water_partition_coefficient,
        'RB': num_rotatable_bonds,
        'AP': aromatic_proportion
    }
    return entry

def get_aromatic_proportion(mol):
    are_aromatic = sum([mol.GetAtomWithIdx(i).GetIsAromatic() for i in range(mol.GetNumAtoms())])
    are_heavy = Descriptors.HeavyAtomCount(mol)
    print(are_aromatic, '/', are_heavy)
    return are_aromatic / are_heavy


display(sample_mol)
print(get_predictors(sample_mol))

pd.DataFrame??

### Calculating Predictors
- cLogP (Octanol-water partition coefficient)
- MW (molecular weight)
- RB (number of rotatable bonds)
- AP (Aromatic proportion = number of aromatic atoms / number of heavy atoms)